In [106]:
# ## Финальное задание:
# - chunking function for 3 types of n-grams (к примеру не + какая-то часть речи или NP или сущ.+ наречие и тд)
# - 3 шаблона для записи в словарь в прошлую дз (такие, как в первом пункте)
# ##  Пояснение по баллам:
# Балл за объяснение того, почему именно эти группы вы взяли, балл за создание такого рода чанкера, балл за за встраивание функции в программу из предыдущей домашки, балл за сравнение качества предсказания тональности с улучшением и без.

In [3]:
from nltk.tokenize import word_tokenize 
import string
import pandas as pd
from pymorphy2 import MorphAnalyzer
#import conllu
import spacy
from spacy import displacy
from collections import Counter
from pymystem3 import Mystem
import re
from nltk.util import ngrams
import requests
from pprint import pprint
import requests
import time
import pandas as pd
import nltk
from nltk.stem import WordNetLemmatizer
from nltk.tokenize import RegexpTokenizer
import pymorphy2
import time
import os
import random
import requests
from bs4 import BeautifulSoup
import pandas as pd
from fake_useragent import UserAgent
from tqdm import tqdm
import math
import numpy as np
import string
import nltk
from nltk.tokenize import wordpunct_tokenize
from nltk.corpus import stopwords
from pymorphy2 import MorphAnalyzer
from tensorflow.keras.preprocessing.text import Tokenizer
import matplotlib.pyplot as plt
from tensorflow.keras.preprocessing.sequence import pad_sequences
from sklearn.model_selection import train_test_split
from tensorflow.keras.models import Sequential
from tensorflow.keras import layers as layer
from tensorflow.keras.callbacks import ModelCheckpoint
from tensorflow.keras import utils
from time import sleep
from datetime import datetime
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn import metrics
from sklearn.model_selection import train_test_split   
import matplotlib.pyplot as plt
from sklearn.model_selection import cross_val_score
from sklearn.svm import SVC
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from collections import defaultdict

In [54]:
!pip install stanza

Looking in indexes: https://pypi.org/simple, https://pypi.ngc.nvidia.com


[notice] A new release of pip available: 22.3.1 -> 23.2.1
[notice] To update, run: C:\Users\User\AppData\Local\Programs\Python\Python311\python.exe -m pip install --upgrade pip



     -------------------------------------- 865.2/865.2 kB 4.5 MB/s eta 0:00:00
     ------------------------------------- 358.9/358.9 kB 21.8 MB/s eta 0:00:00
     ------------------------------------- 430.5/430.5 kB 26.3 MB/s eta 0:00:00
     ------------------------------------- 172.3/172.3 MB 15.9 MB/s eta 0:00:00
     ---------------------------------------- 5.7/5.7 MB 28.1 MB/s eta 0:00:00
     ---------------------------------------- 2.1/2.1 MB 43.9 MB/s eta 0:00:00
     ---------------------------------------- 133.1/133.1 kB ? eta 0:00:00
     ---------------------------------------- 536.2/536.2 kB ? eta 0:00:00


In [41]:
!python -m spacy download ru_core_news_md

Looking in indexes: https://pypi.org/simple, https://pypi.ngc.nvidia.com
     ---------------------------------------- 41.9/41.9 MB 2.8 MB/s eta 0:00:00
✔ Download and installation successful
You can now load the package via spacy.load('ru_core_news_md')



[notice] A new release of pip available: 22.3.1 -> 23.2.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [45]:
! pip install natasha

Looking in indexes: https://pypi.org/simple, https://pypi.ngc.nvidia.com

  DEPRECATION: docopt is being installed using the legacy 'setup.py install' method, because it does not have a 'pyproject.toml' and the 'wheel' package is not installed. pip 23.1 will enforce this behaviour change. A possible replacement is to enable the '--use-pep517' option. Discussion can be found at https://github.com/pypa/pip/issues/8559

In [7]:
nlp = spacy.load('ru_core_news_md')

In [4]:
m = MorphAnalyzer()
my = Mystem()

## Описание текста, выбранного для анализа
Для анализа была взята поэма "Повесть каменного века". В тексте много примеров, сложных для анализа. Примеры:
- Омонимы (неполные омографы): "зари" - в значении ед.ч. род.п. и мн.ч. им.п.
- Причастия/прилагательные: "преданной" (преданной анафеме/преданной любимым)
- Сложные слова с дефисным написанием: "рок-судья"
- Окказионализмы (выдуманные слова): "шестопером" - и другие слова, которые могут не содержаться в "обучающей" выборке парсера (архаизмы - "ея")
## Какой тегсет использовать?
В качестве тегсета был использован тэгсетов Universal Dependencies - один из самых широко используемых среди всех тегсетов. Во-первых, его примуществом является тот факт, что он используется во многих библиотеках. Во-вторых, некоторые аспекты этого тэгсета помогают обнаружить проблемы разметки (в том числе другими тэгсетами). Например, он разделяет союзы, используемые в сложносочиненных предложениях (CCONJ) и сложноподчиненных предложениях (SCONJ). Даже тэгстет mystem, который разработан специально для русского языка, не обладает такими хараткеристиками. 

In [17]:
file = open(r'C:\Users\User\PycharmProjects\practice\dep2label\«Где И.txt', 'r', encoding="utf-8")

In [10]:
spis = []
with open(r'C:\Users\User\PycharmProjects\practice\dep2label\«Где И.txt','r', encoding="utf-8") as fp:
    for line in fp:
        spis.append(line.rstrip('\n'))

In [11]:
token_spis = []
for i in spis:
    token_spis.append(word_tokenize(i))

In [12]:
tokens = []
df = pd.DataFrame(columns = {'tokens'})
for spis in token_spis:
    for el in spis:
        if el not in list(string.punctuation):
            tokens.append(el)
df['tokens'] = tokens

In [8]:
df.to_excel('corpus.xlsx', encoding = 'UTF-8')


KeyboardInterrupt



In [13]:
document = nlp(file.read())

In [18]:
doc_stem = my.analyze(file.read())

In [19]:
stem_data = pd.DataFrame()
stem_tags = []
stem_words = []
for w in doc_stem:
    if 'analysis' in w.keys() and len(w['analysis'])!=0:
        stem_tags.append(re.split("=|,| ", w['analysis'][0]['gr'])[0])
        stem_words.append(str(w['text']).lower())
stem_data['tokens'] = stem_words
stem_data['tags'] = stem_tags

In [20]:
stem_data = stem_data.rename(columns={'tags': 's_tags'})
stem_data.head()

,tokens,s_tags
0,где,ADVPRO
1,и,S
2,в,PR
3,лесу,S
4,дремучем,A


In [21]:
morph_data = pd.DataFrame()
morph_tags = []
morph_words = []
for word in tokens:
   morph_tags.append(m.parse(word)[0].tag), morph_words.append(m.parse(word)[0].word)
morph_data['tags']=morph_tags
morph_data['tokens']=morph_words

In [22]:
morph_data.head()

,tags,tokens
0,PNCT,«
1,"ADVB,Ques",где
2,CONJ,и
3,PREP,в
4,"NOUN,inan,masc sing,loc2",лесу


In [23]:
def only_pos(row):
    if ',' in str(row['tags']):
        return str(row['tags']).split(',')[0]
    else:
        return str(row['tags'])
morph_data['tags'] = morph_data.apply(only_pos, axis=1)

In [24]:
morph_data = morph_data.rename(columns={'tags': 'm_tags'})
morph_data.head()

,m_tags,tokens
0,PNCT,«
1,ADVB,где
2,CONJ,и
3,PREP,в
4,NOUN,лесу


In [15]:
spacy_data = pd.DataFrame()
sp_tokens = []
sp_tags = []
for token in document:
    sp_tokens.append(str(token).lower())
    sp_tags.append(token.pos_)
spacy_data['tokens'] = sp_tokens
spacy_data['tags_sp'] = sp_tags

In [16]:
spacy_data.head()

,tokens,tags_sp
0,«,PUNCT
1,где,ADV
2,и,PART
3,?,PUNCT
4,\n,SPACE


In [93]:
spacy_data = spacy_data[spacy_data['tags_sp']!='PUNCT']
morph_data = morph_data[morph_data['m_tags']!='PNCT']

In [95]:
spacy_data = spacy_data[spacy_data['tags_sp']!='SPACE']

In [94]:
morph_data.to_excel('morph.xlsx')
stem_data.to_excel('stem.xlsx')

In [96]:
spacy_data.to_excel('spacy.xlsx')

In [87]:
data = pd.read_excel(r'C:\Users\User\PycharmProjects\practice\dep2label\corpus.xlsx')

In [88]:
data['tokens'] = data['tokens'].map(lambda x: str(x).lower())
#data = data.drop(['Unnamed: 0', 'Unnamed: 3'], axis= 1 , inplace= True )
data

,Unnamed: 0,tokens,POS,Unnamed: 3
0,1,где,ADV,NaN
1,2,и,CCONJ,NaN
2,3,в,ADP,NaN
3,4,лесу,NOUN,NaN
4,5,дремучем,ADJ,NaN
...,...,...,...,...
229,230,ей,PRON,NaN
230,231,в,ADP,NaN
231,232,делах,NOUN,NaN
232,233,ее,PRON,NaN


In [89]:
data = data.drop(['Unnamed: 0', 'Unnamed: 3'], axis= 1)

In [90]:
data_merged =pd.merge(data,spacy_data, on = 'tokens')
data_merged= data_merged.merge(morph_data, on = 'tokens')
data_merged = data_merged.merge(stem_data, on = 'tokens')

In [91]:
data_merged.head(20)

,tokens,POS,tags_sp,m_tags,s_tags
0,где,ADV,ADV,ADVB,ADVPRO
1,где,ADV,ADV,ADVB,ADVPRO
2,где,ADV,ADV,ADVB,ADVPRO
3,где,ADV,ADV,ADVB,ADVPRO
4,где,ADV,ADV,ADVB,ADVPRO
5,где,ADV,ADV,ADVB,ADVPRO
6,где,ADV,ADV,ADVB,ADVPRO
7,где,ADV,ADV,ADVB,ADVPRO
8,где,ADV,ADV,ADVB,ADVPRO
9,где,ADV,ADV,ADVB,ADVPRO


In [55]:
data = data.set_index('tokens').drop_duplicates()
data

,POS,tags_sp,m_tags,s_tags
tokens,,,,
где,ADV,ADV,ADVB,ADVPRO
и,CCONJ,PART,CONJ,S
и,CCONJ,PART,CONJ,CONJ
и,CCONJ,CCONJ,CONJ,S
и,CCONJ,CCONJ,CONJ,CONJ
...,...,...,...,...
длиною,CCONJ,NOUN,NOUN,S
многое,PRON,NOUN,NPRO,SPRO
могуч,ADJ,PROPN,ADJS,A


In [136]:
stem_dict = {'A':'ADJ','ADVPRO':'ADV','ANUM':'ADJ','APRO':'DET','COM':'ADJ','CONJ':'SCONJ','NONLEX':'X','PR':'ADP',
             'S':'NOUN','SPRO':'PRON','UNKN':'X','V':'VERB'}
morph_dict = {'ADVB':'ADV','CONJ':'CCONJ','PREP':'ADP','ADJF':'ADJ',
              'NRPO':'PRON','PRCL':'PART','GRND':'VERB'
              }

In [137]:
df = pd.read_excel(r'C:\Users\User\PycharmProjects\practice\dep2label\corpus.xlsx')

In [138]:
df = df.drop('Unnamed: 0',  axis= 1)

In [141]:
df.head()

,tokens,POS,tokens.1,m_tags,tokens.2,tags_sp,tokens.3,s_tags
0,Где,ADV,где,ADV,где,ADV,где,ADV
1,И,CCONJ,и,CCONJ,и,PART,и,NOUN
2,В,ADP,в,ADP,в,ADP,в,ADP
4,дремучем,ADJ,дремучем,ADJ,дремучем,ADJ,дремучем,ADJ
8,Свои,PRON,свои,ADJ,свои,DET,свои,DET


In [140]:
df = (df
      .query("m_tags in @morph_dict.keys()")
      .assign(m_tags=lambda x: x["m_tags"].replace(morph_dict)))
df = (df
      .query("s_tags in @stem_dict.keys()")
      .assign(s_tags=lambda x: x["s_tags"].replace(stem_dict)))

In [142]:
def comparison(stand, comp):
    counter = 0 
    for s, c in zip(stand,comp):
        if s == c:
            counter+=1
        # else:
        #     print(s,c)
    return round(counter/len(stand),4)

In [143]:
print('Accuracy for MorphAnalyzer:',comparison(list(df['POS'].values),list(df['m_tags'].values)))
print('Accuracy for Mystem:',comparison(list(df['POS'].values),list(df['s_tags'].values)))
print('Accuracy for Spacy:',comparison(list(df['POS'].values),list(df['tags_sp'].values)))

Accuracy for MorphAnalyzer: 0.8514
Accuracy for Mystem: 0.7838
Accuracy for Spacy: 0.8378


## Находим шаблоны для chunking
Чтобы понять, какие n-gramm надо доставать из текстов, чтобы определить их тональность, рассмотрим биграммы, которые встречаются в текстах с уже определенной положительной и отрицательной тональностями.

In [5]:
session = requests.session()
ua = UserAgent(verify_ssl=False)

In [6]:
# информация из страницы со списком новостей
def parse_news_page_block(one_block):
    block = {}
    ua = UserAgent(verify_ssl=False)
    a = one_block.find('a')
    block['href'] = a.attrs['href']
    return block
# отдельная страница новости   
def parse_one_article(block):
    ua = UserAgent(verify_ssl=False)
    url_one = 'https://spasibovsem.ru' + block['href']
    block['page_id'] = block['href']
    req = session.get(url_one, headers={'User-Agent': ua.random})
    time.sleep(2)
    page = req.text
    soup = BeautifulSoup(page, 'html.parser')
    block['full_text'] = soup.find("div", {"class": "text response-text description"}, {"itemprop": "reviewBody"}).text   
    block['rating'] = (str(soup.find("abbr", {"class": "rating"}))).split('>')[0].split()[2].split('"')[1]
    return block 
def get_nth_page(page_number):
    # скачиваем
    url =  f'https://spasibovsem.ru/filmy-otzyvy/?page={page_number}'
    req = session.get(url, headers={'User-Agent': ua.random})
    time.sleep(2)
    page = req.text
    soup = BeautifulSoup(page, 'html.parser')
    
    # находим новости
    news = soup.find_all('div', {'class': 'response-name'})
    
    # идем по новостям и обрабатываем их
    blocks = []
    for n in news:
        try:
            blocks.append(parse_news_page_block(n))
        except Exception as e:
            print(e)
    seen_news = []
    # идем по отдельным статьям и достаем информацию
    result = []
    for b in blocks:
        try:
            res = parse_one_article(b)
            res['id'] = b['page_id']
            result.append(res)
        except Exception as e:
            print(e)
    # возвращаем найденную информацию
    return result

In [7]:
def run_all(n_pages):
    reses = []
    for i in tqdm(range(n_pages)):
        blocks = get_nth_page(i+1)
        reses.append(blocks)
    return reses

In [9]:
spis = run_all(30)

  0%|          | 0/30 [00:00<?, ?it/s]

Invalid return character or leading space in header: User-Agent


 27%|██▋       | 8/30 [14:21<38:43, 105.59s/it]

Invalid return character or leading space in header: User-Agent


 30%|███       | 9/30 [15:57<35:53, 102.54s/it]

...

 90%|█████████ | 27/30 [57:14<07:59, 159.67s/it]

Invalid return character or leading space in header: User-Agent


 93%|█████████▎| 28/30 [59:52<05:18, 159.19s/it]

Invalid return character or leading space in header: User-Agent


100%|██████████| 30/30 [1:05:03<00:00, 130.12s/it]


In [10]:
rev = pd.DataFrame(columns =['texts', 'rating', 'tonality'])
texts = []
ratings = []
for page in spis:
    for slovik in page:
        texts.append(slovik['full_text'])
        ratings.append(slovik['rating'])
rev['texts'] = texts
rev['rating'] = ratings
tone_dictionary ={'1' : 0, '2' : 0, '4' : 1, '5' : 1, '3' : 0.5}
#df[df['rating']=='3']['tonality'] = 0.5
rev['tonality'] = rev['rating'].map(tone_dictionary)
rev = rev.loc[rev['tonality'] != 0.5]

In [11]:
stops = stopwords.words("russian")

In [12]:
reviews_preprocessed = []
for a_text in list(rev['texts'].values):
    a_tokens = wordpunct_tokenize(a_text)
    a_lemmatized = " ".join([m.parse(item)[0].normal_form.lower() for item in a_tokens])
    reviews_preprocessed.append(a_lemmatized)

In [14]:
n=2
slovik_pos = defaultdict(int)
slovik_neg = defaultdict(int)
for text, tone in zip(reviews_preprocessed, list(rev['tonality'].values)):
    if tone == 0:
        tokens = nltk.word_tokenize(text)
        bigrams = ngrams(tokens, n, pad_left=True, pad_right=True)
        for bigram in bigrams:
            slovik_neg[bigram] += 1
    if tone == 1:
        tokens = nltk.word_tokenize(text)
        bigrams = ngrams(tokens, n, pad_left=True, pad_right=True)
        for bigram in bigrams:
            slovik_pos[bigram] += 1  

In [15]:
sloviki = [slovik_neg, slovik_pos]
for slovik in sloviki:
    for k, v in slovik.copy().items():
        if v<=2:
            del slovik[k]

In [17]:
sorted_pos = defaultdict(int)
for el,c in slovik_pos.items():
    if el[0] not in list(string.punctuation) and el[1] not in list(string.punctuation):
        sorted_pos[el]=c
sorted_pos = sorted(sorted_pos.items(), key=lambda x: x[1],reverse=True)

In [18]:
sorted_neg = defaultdict(int)
for el,c in slovik_neg.items():
    if type(el) is not int and el[0] not in list(string.punctuation) and el[1] not in list(string.punctuation):
        sorted_neg[el]=c
sorted_neg = sorted(sorted_neg.items(), key=lambda x: x[1],reverse=True)

## Выбор н-грамм
В прошлый раз большинство ошибок касались разметки негативных отзывов (из-за несбалансированной выборки), поэтому шаблоны мы будем извлекать для классификации (пополнения словаря) негативных отзывов. 

Можно заметить, что чаще всего в биграммах встречается частица "не", особенно частотными являются биграммы с "не"+глагол. Также часто встречаются биграммы союз + "не". Кажется, что негативные отзывы должны содержать прилагательное "плохой", поэтому также используем биграмму "плохой" + Noun (т.к. чаще всего прилагательные сочетаются с существительными, и мы предполагаем, что между ними не будет расстояния,т.е. других словоформ).

In [44]:
def chunker(text):
    text_sp = text.split()
    counter = 0
    for i in range(len(text_sp)):
        try:
            if text_sp[i] == 'не' and text_sp[i+1] not in list(string.punctuation) and str(m.parse(text_sp[i+1])[0].tag)=='VERB':
                counter+=1
            if text_sp[i] not in list(string.punctuation) and str(m.parse(text_sp[i])[0].tag)=='CONJ' and text_sp[i+1] == 'не':
                counter+=1
            if text_sp[i]=='плохой' and text_sp[i+1] not in list(string.punctuation) and str(m.parse(text_sp[i+1])[0].tag)=='NOUN':
                counter+=1
        except Exception:
            pass
    return counter
#m.parse(word)[0].tag

In [20]:
#1-gramms
one_pos = {}
one_neg = {}
for text, tone in zip(reviews_preprocessed, list(rev['tonality'].values)):
    if tone == 0:
        for word in text.split():
            if word not in stops and word not in string.punctuation and word not in one_neg:
                one_neg[word] = 1
            elif word not in stops and word not in string.punctuation and word in one_neg:
                one_neg[word] += 1
    if tone == 1:
        for word in text.split():
            if word not in stops and word not in string.punctuation and word not in one_pos:
                one_pos[word] = 1
            elif word not in stops and word not in string.punctuation and word in one_pos:
                one_pos[word] += 1  
sloviki = [one_neg, one_pos]
for slovik in sloviki:
    for k, v in slovik.copy().items():
        if v<=2:
            del slovik[k]
one_neg_set = []
one_pos_set = []
for k in one_neg.keys():
    if k not in list(one_pos.keys()):
        one_neg_set.append(k)
for k in one_pos.keys():
    if k not in list(one_neg.keys()):
        one_pos_set.append(k)

In [21]:
def run_test(n_pages):
    reses = []
    for i in tqdm(range(n_pages)):
        blocks = get_nth_page(i+100)
        reses.append(blocks)
    return reses
test_spis = run_test(8)

  0%|          | 0/8 [00:00<?, ?it/s]

Invalid return character or leading space in header: User-Agent


 12%|█▎        | 1/8 [03:06<21:46, 186.61s/it]

Invalid return character or leading space in header: User-Agent


 25%|██▌       | 2/8 [06:01<17:59, 179.88s/it]

Invalid return character or leading space in header: User-Agent


 75%|███████▌  | 6/8 [17:37<05:46, 173.21s/it]

Invalid return character or leading space in header: User-Agent


100%|██████████| 8/8 [24:23<00:00, 182.99s/it]


In [22]:
test = pd.DataFrame(columns =['texts', 'rating', 'tonality'])

In [23]:
clus = [test_spis]
texts = []
ratings = []
for spis in clus:
    for page in spis:
        for slovik in page:
            texts.append(slovik['full_text'])
            ratings.append(slovik['rating'])

In [24]:
test['texts'] = texts
test['rating'] = ratings
test['tonality'] = test['rating'].map(tone_dictionary)
test = test.loc[test['tonality'] != 0.5]
reviews_test_preprocessed = []
for a_text in list(test['texts'].values):
    a_tokens = wordpunct_tokenize(a_text)
    a_lemmatized = " ".join([m.parse(item)[0].normal_form.lower() for item in a_tokens])
    reviews_test_preprocessed.append(a_lemmatized)

In [45]:
res_tone = []
errors = 0
for text in reviews_test_preprocessed:
    neg = 0
    pos = 0
    neg+=chunker(text)
    for word in text.split():
        if word not in stops and word not in string.punctuation and word in one_neg_set:
            neg+=1
        elif word not in stops and word not in string.punctuation and word in one_pos_set:
            pos+=1
    if neg>pos:
        res_tone.append(0)
    elif pos>neg:
        res_tone.append(1)
    else:
        errors+=1
print(errors)

0


In [46]:
print(round(sum(1 for x,y in zip(res_tone, test['tonality']) if x == y) / len(res_tone),2))

0.89


In [27]:
res_one = []
errors = 0
for text in reviews_test_preprocessed:
    neg = 0
    pos = 0
    for word in text.split():
        if word not in stops and word not in string.punctuation and word in one_neg_set:
            neg+=1
        elif word not in stops and word not in string.punctuation and word in one_pos_set:
            pos+=1
    if neg>pos:
        res_one.append(0)
    elif pos>neg:
        res_one.append(1)
    else:
        res_one.append(0.5)
        errors+=1
print(errors)

0


In [30]:
accuracy_without_bi = sum(1 for x,y in zip(res_one, test['tonality']) if x == y) / len(res_one)

In [37]:
print(accuracy_without_bi)

0.8623076923076923


результат: точность выросла на 3%